# What is Transfer learning?

Transfer learning is a technique in machine learning in which knowledge learned from a task is re-used in order to boost performance on a related task. For example, for image classification, knowledge gained while learning to recognize cars could be applied when trying to recognize trucks. Wikipedia

## Why use transfer learning?
1. Can leverage an existing neural network architecture proven to work on problems similar to our own
2. Can leverage a woring network architecture which has already learned patterns on similar data to our own (so great performence with low data)

In [1]:
import torch
import torchvision
torch.__version__,torchvision.__version__

('2.1.0.dev20230713', '0.16.0.dev20230713')

Now we've got the versions of torch and torchvision, we're after, let's import the code we've writte in previous section 

In [2]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from going_modular.going_modular import data_setup, engine
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !mv pytorch-deep-learning/data/pizza_steak_sushi_20_percent.zip ./data/05
    !unzip ./data/05/pizza_steak_sushi_20_percent.zip
    # !rm -rf pytorch-deep-learning
    from going_modular.going_modular import data_setup, engine

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
!nvidia-smi

Fri Jul 14 20:32:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:26:00.0  On |                  N/A |
|  0%   48C    P8    12W / 170W |    599MiB / 12288MiB |     23%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##  Get data

We need our pizza, steak, sushi data to build a transfer learning model

In [5]:
# !unzip ./data/05/pizza_steak_sushi_20_percent.zip

In [6]:
train_dir = "./data/05/train/"
test_dir = "./data/05/test/"

## Create Datasets and Dataloaders

Now've got some data, now we wanna turn it into PyTorch DataLoaders.

We can use `data_setup.py`and `create_dataloaders()`

Methods of transformations:
1. Manual
2. Automatically - the transforms are picked by pretrained model

When using a pretrained model, its important that the data that you pass through is transformed in the same way that the data was trained on

In [7]:
# Pay attension when using pre trained models
from going_modular.going_modular import *

### Create transforms manually 

In [8]:
from torchvision import transforms
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [9]:
manual_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    normalize
])

In [10]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,test_dir,manual_transforms,32)

### Create transforms automatically

In [11]:
import torchvision

In [12]:
weights = torchvision.models.EfficientNet_V2_M_Weights.IMAGENET1K_V1 # Default = best weight

In [13]:
weights

EfficientNet_V2_M_Weights.IMAGENET1K_V1

In [14]:
# Get the transforms used to create our pretrained weights
auto_transforms= weights.transforms()

In [15]:
auto_transforms

ImageClassification(
    crop_size=[480]
    resize_size=[480]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [16]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir,test_dir,auto_transforms,32)

## Getting a Pretrained model

### Which pretrained model should you use?

*Experiment, experiment, experiment!*

The whole idea of transfer learning is to take an already well performing model from a problem space similar to your own and then customize to your own problem.

Three things to consider:
1. Speed - how fast does it run?
2. Size - how big is it?
3. Performance - how accuracte is it?

Where would the model live?

### Setting up a pretrained model

In [17]:
model = torchvision.models.efficientnet_v2_m(weights)

/home/user/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /home/user/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 208M/208M [02:13<00:00, 1.63MB/s]


In [19]:
# model

In [21]:
model.avgpool

AdaptiveAvgPool2d(output_size=1)

In [27]:
# model.classifier[1] = torch.nn.Linear(1280,len(class_names),bias=True)

### Getting a summary of our model with torchinfo

In [31]:
from torchinfo import summary

In [35]:
summary(model,input_size=(1,3,224,224),col_names=['input_size','output_size','num_params','trainable'],col_width=20,row_settings=['var_names'])

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 3]               --                   True
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 24, 112, 112]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 24, 112, 112]    648                  True
│    │    └─BatchNorm2d (1)                                  [1, 24, 112, 112]    [1, 24, 112, 112]    48                   True
│    │    └─SiLU (2)                                         [1, 24, 112, 112]    [1, 24, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 24, 112, 112]    [1, 24, 112,